<a href="https://colab.research.google.com/github/NateMophi/SCC-454/blob/main/LAB2/SCC454_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark==4.0.0 -q

# Java Installation
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [ ]:
# Set Java environmenr variable
import os
os.environ["JAVA HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print("PySpark & Java installed successfully!")

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Session Creation
spark = SparkSession.builder.appName("SCC454-SparkIntro").config("spark.driver.memory", "4g")\
        .config("spark.ui.port", "4050")\
        .getOrCreate()

  # Underlying Context
sc = spark.sparkContext
print(f"Spark Version: {spark.version}")
print(f"Spark App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")

# **Resilient Distributed Datasets (RDDs)**

In [ ]:
# RDD Creation

# 1a) Parallelization from a Python Collection
nums = [1,2,3,4,5,6,7,8,9,10]
nums_rdd =sc.parallelize(nums)

type(nums_rdd)
print(f"Number of paritions: {nums_rdd.getNumPartitions()}")
print(f"First 5 elements : {nums_rdd.take(5)}")



In [ ]:
# 1b) Parallelize with a specific number of partitions
nums_rdd_4part = sc.parallelize(nums, 4)
print(f"Number of partitions: {nums_rdd_4part.getNumPartitions()}")

# View Partitions
print("\nData in each partition: ")
print(nums_rdd_4part.glom().collect())

In [ ]:
# 2) Create RDD from text file
sample_text = """Apache Spark is a unified analytics engine for large-scale data processing.
It provides high-level APIs in Java, Scala, Python and R.
Spark powers a stack of libraries including SQL and DataFrames.
It also includes MLlib for machine learning and GraphX for graph processing.
Spark can run on Hadoop, Apache Mesos, Kubernetes, standalone, or in the cloud."""


with open("spark_intro.txt", "w") as f:
  f.write(sample_text)

# Load text file as RDD
text_rdd = sc.textFile("spark_intro.txt")
print(f"Number of lines: {text_rdd.count()}")
print("\n1st 3 lines:")
for line in text_rdd.take(3):
  print(f" - {line}")

In [ ]:
!git add .


In [ ]:
!git commit -m "naught"

On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)


In [ ]:
!git push

error: src refspec refs/heads/main does not match any
error: failed to push some refs to 'https://github.com/NateMophi/SCC-454.git'
